In [ ]:
import pandas as pd
import numpy as np
# import nltk
# from nltk.corpus import stopwords
# # from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
# import re
# import torch
# import torchtext
# from typing import Iterable
# from torchtext.data.utils import get_tokenizer
# import underthesea
# from torchtext.vocab import build_vocab_from_iterator
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

d:\python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\python310\lib\site-packages\torchtext\data\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upg

In [3]:
df = pd.read_csv('Sentence pairs in English-Vietnamese - 2025-11-12.tsv', sep='\t')
df.columns = ['eng_id', 'english', 'vie_id', 'vietnamese']
df = df[['english', 'vietnamese']]
df.head()

,english,vietnamese
0,Today is June 18th and it is Muiriel's birthday!,"Hôm nay là ngày 18 tháng sáu, và cũng là ngày ..."
1,Muiriel is 20 now.,Bây giờ Muiriel được 20 tuổi.
2,"The password is ""Muiriel"".","Mật mã là ""Muiriel""."
3,I'm at a loss for words.,Tôi hết lời để nói.
4,I'm at a loss for words.,Tôi không biết nói gì.


In [4]:
df['eng_tokens'] = df['english'].apply(nltk.word_tokenize) 
df['vie_tokens'] = df['vietnamese'].apply(underthesea.word_tokenize) 
df.head()

,english,vietnamese,eng_tokens,vie_tokens
0,Today is June 18th and it is Muiriel's birthday!,"Hôm nay là ngày 18 tháng sáu, và cũng là ngày ...","[Today, is, June, 18th, and, it, is, Muiriel, ...","[Hôm nay, là, ngày, 18, tháng, sáu, ,, và, cũn..."
1,Muiriel is 20 now.,Bây giờ Muiriel được 20 tuổi.,"[Muiriel, is, 20, now, .]","[Bây giờ, Muiriel, được, 20, tuổi, .]"
2,"The password is ""Muiriel"".","Mật mã là ""Muiriel"".","[The, password, is, ``, Muiriel, '', .]","[Mật mã, là, "", Muiriel, "", .]"
3,I'm at a loss for words.,Tôi hết lời để nói.,"[I, 'm, at, a, loss, for, words, .]","[Tôi, hết, lời, để, nói, .]"
4,I'm at a loss for words.,Tôi không biết nói gì.,"[I, 'm, at, a, loss, for, words, .]","[Tôi, không, biết, nói, gì, .]"


In [5]:
SRC_LANGUAGE = 'en'
TGT_LANGUAGE = 'vi'

def data_iterator(df) -> Iterable:
    for _, row in df.iterrows():
        yield (row['english'], row['vietnamese'])


In [10]:
token_transform = {}
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')
token_transform[TGT_LANGUAGE] = lambda x: underthesea.word_tokenize(x, format='text').split()

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

vocab_transform = {}

def yield_tokens(data_iter: Iterable, language: str):
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}
    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[ln] = build_vocab_from_iterator(
        yield_tokens(data_iterator(df), ln),
        min_freq=1,
        specials=special_symbols,
        special_first=True
    )
    vocab_transform[ln].set_default_index(UNK_IDX)

Q2